In [1]:
import pandas as pd
import json, os, time
import requests, base64, hmac, threading
from datetime import datetime, timedelta

In [ ]:
cur_path = os.getcwd()
def save_file(df, instrument_id, DATA_FOLDER, DATA_DIRECTORY='SWAP', file_name=None):
    path = os.path.join(cur_path,DATA_DIRECTORY, DATA_FOLDER)
    os.makedirs(path, exist_ok=True)
    if not file_name:
        file_name=f'{instrument_id}_{DATA_FOLDER}.csv'
    file_path = os.path.join(path, file_name)
    df.to_csv(file_path)

In [3]:
def crawl_his_ohlc(instrument_id, timeframe, limit,after=None):
    url="https://www.okx.com/api/v5/market/candles"
    params = {'instId': instrument_id, 'bar': timeframe, 'limit': limit}
    if after: params['after'] = after
    response = requests.get(url, params=params)
    data = response.json().get('data', [])
    return data

In [4]:
end=datetime.now()
start = end - timedelta(days=300)
def crawl_his_ohlc_full(start_date, end_date):
    ohlc = []
    start_ts = int(start.timestamp() * 1000)
    init = crawl_his_ohlc('BTC-USDT-SWAP', '1H',300,None)
    oldest=int(init[-1][0])
    ohlc.extend(init)
    #cur_ts = int(oldest.timestamp() * 1000) - 1
    while oldest > start_ts:
            data = crawl_his_ohlc('BTC-USDT-SWAP', '1H',300,oldest)

            if not data:
                print('empty data')
                break
            #df = df[df.index >= start_date]
            ohlc.extend(data)
            oldest = int(data[-1][0])
            #cur_ts = int(oldest.timestamp() * 1000)

            print(f"get data to {pd.to_datetime(oldest,unit='ms').strftime('%Y-%m-%d')}")
            time.sleep(0.5) # condition avoid ratelimit
    d1=pd.to_datetime(pd.to_numeric(ohlc[-1][0]),unit='ms').strftime("%Y_%m_%d")
    d2=pd.to_datetime(pd.to_numeric(ohlc[0][0]),unit='ms').strftime("%Y_%m_%d")
    path=f'/mnt/d/learn/DE/Semina_project/backend/extract/data/raw/crawl_ohlc_from{d1}_to_{d2}.json'
    with open(path, "w", encoding="utf-8") as f:
        json.dump(ohlc, f, ensure_ascii=False, indent=2)
    print(f"Extracted {len(ohlc)} regions and exchanges.")
crawl_his_ohlc_full(start,end)



get data to 2025-10-27
get data to 2025-10-15
get data to 2025-10-02
get data to 2025-09-22
empty data
Extracted 1440 regions and exchanges.


In [ ]:
def get_long_short(currency,period):
    url = "https://www.okx.com/api/v5/rubik/stat/contracts/long-short-account-ratio"
    params = {"ccy": currency, "period": period}
    r = requests.get(url, params=params)
    r.raise_for_status()
    data = r.json().get('data', [])
    d1=pd.to_datetime(pd.to_numeric(data[-1][0]),unit='ms').strftime("%Y_%m_%d")
    d2=pd.to_datetime(pd.to_numeric(data[0][0]),unit='ms').strftime("%Y_%m_%d")
    path=f'/mnt/d/learn/DE/Semina_project/backend/extract/data/raw/oi/crawl_long_short_ratio_from{d1}_to_{d2}.json'
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"Extracted {len(data)} regions and exchanges.")
get_long_short('BTC','1H')

In [ ]:
def get_open_interest(currency,period):
    url = "https://www.okx.com/api/v5/rubik/stat/contracts/open-interest-volume"
    params = {"ccy": currency, "period": period}
    r = requests.get(url, params=params)
    r.raise_for_status()
    data = r.json().get('data', [])
    d1=pd.to_datetime(pd.to_numeric(data[-1][0]),unit='ms').strftime("%Y_%m_%d")
    d2=pd.to_datetime(pd.to_numeric(data[0][0]),unit='ms').strftime("%Y_%m_%d")
    path=f'/mnt/d/learn/DE/Semina_project/backend/extract/data/raw/oi/crawl_open_interest_from{d1}_to_{d2}.json'
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"Extracted {len(data)} regions and exchanges.")
get_open_interest('BTC','1H')

In [ ]:
"""Lấy dữ liệu Taker Buy/Sell Volume."""
url = "https://www.okx.com/api/v5/rubik/stat/taker-volume"
params = {"ccy": 'BTC', "instType": "SPOT", "period": '1H'}
r = requests.get(url, params=params)
data = r.json().get('data', [])
data


In [18]:
def  get_open_interest_data(url, params, name):
    r = requests.get(url, params=params)
    r.raise_for_status()
    data = r.json().get('data', [])
    d1=pd.to_datetime(pd.to_numeric(data[-1][0]),unit='ms').strftime("%Y_%m_%d")
    d2=pd.to_datetime(pd.to_numeric(data[0][0]),unit='ms').strftime("%Y_%m_%d")
    path=f'/mnt/d/learn/DE/Semina_project/backend/extract/data/raw/oi/crawl_{name}_from{d1}_to_{d2}.json'
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"Extracted {len(data)} regions and exchanges.")
def full_open_interest_data(currency,period):
    api_url=[
        (
            "https://www.okx.com/api/v5/rubik/stat/contracts/open-interest-volume",
            {"ccy": currency, "period": period},
            'open-interest-volume'
        ),
        (
            "https://www.okx.com/api/v5/rubik/stat/contracts/long-short-account-ratio",
            {"ccy": currency, "period": period},
            'long-short-account-ratio'
        ),
        (
            "https://www.okx.com/api/v5/rubik/stat/taker-volume",
            {"ccy": currency, "instType": "SPOT", "period": period},
            'taker-volume'
        )
    ]
    for url,params,name in api_url:
        get_open_interest_data(url, params, name)
full_open_interest_data('BTC','1H')

Extracted 719 regions and exchanges.
Extracted 719 regions and exchanges.
Extracted 719 regions and exchanges.
